## __Prerequisites__

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from gensim.models.word2vec import Word2Vec
from tqdm import tqdm
import time

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from eunjeon import Mecab
import re

nltk.download('punkt')
#nltk.download('stopwords')


import pickle
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer


from keras_preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical


from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

def get_scores(y_test, predicted):
    print('-------------------------')
    print('Accuracy_score = ', accuracy_score(y_test, predicted))
    print('precision_score = ', precision_score(y_test, predicted))
    print('recall_score = ', recall_score(y_test, predicted))
    print('f1_score = ', f1_score(y_test, predicted))
    print('-------------------------\n')
    
    confusion_mat = confusion_matrix(y_test, predicted)
    print(confusion_mat)
    

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\vaiv\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## __Preprocessing__

- 아래에 작성된 data는 뉴스 기사를 크롤링한 데이터입니다.
- data의 column은 date, writerName(신문사), title(기사제목), content(기사내용), tag(binary)로 구성되어있습니다.

In [2]:
class DataPreprocessing:
    # Get writer_score(ratio of tag_1) : 신문사가 tag==1인 기사를 얼마나 작성하였는지 비율 계산
    def get_writer_score(self, data, train=False):
        if train == True:
            df_writer = data[['writerName', 'tag']].groupby('writerName', as_index=False).sum() # 신문사별 tag==1 개수 추출
            df_writer.rename(columns={'tag':'tag_1'}, inplace=True)

            total_cnt = data[['writerName', 'tag']].groupby(['writerName'], as_index=False).count()['tag'] # 신문사별 기사 개수 추출
            df_writer['total_cnt'] = total_cnt

            df_writer['writer_score'] = df_writer['tag_1']/df_writer['total_cnt'] # 신문사별 tag==1인 비율 계산
            
            df_writer.to_csv('../data/df_writer.csv', index=False)
        else:
            return

    # 기존 DataFrame에 writer_score(신문사별 tag==1인 기사 작성 비율)를 merge
    def merge_writer_score(self, data):
        df_writer = pd.read_csv('../data/df_writer.csv')
        data = pd.merge(data, df_writer[['writerName', 'writer_score']], how='left', on='writerName') # writerName 기준으로 merge
        new_data = data[['date', 'writer_score', 'title', 'content', 'tag']]
        return new_data
    
    # 기사 제목을 tf-idf로 벡터화 
    def title_tf_idf(self, data, target):
        # 1. Mecab 객체 선언
        mecab = Mecab()
        
        # 2. Get nouns from df['title']
        x_data = data['title'].apply(lambda x: ' '.join(mecab.nouns(x)))
        
        # 3-1. Get vector count(By CountVectorizing)
        # count_vect = CountVectorizer()
        # X_counts = count_vect.fit_transform(x_data)
        
        # 3-2. Get vector count(By TfidfVectorizing)
        tfidf_vect = TfidfVectorizer()
        X_counts = tfidf_vect.fit_transform(x_data)
        
        # 4. Save word vector
        #pickle.dump(count_vect.vocabulary_, open("count_vector.pkl","wb"))

        # 5. Transform word vector to ti-idf
        tfidf_transformer = TfidfTransformer()
        X_tfidf = tfidf_transformer.fit_transform(X_counts)
        df_tfidf = pd.DataFrame(X_tfidf.toarray())
        # df_tfidf['writer_score'] = data['writer_score'] # 추후 분석 시 writer_score를 포함할 경우 주석 제거
        df_tfidf.columns = list(map(str, list(df_tfidf.columns)))
        # display(df_tfidf)
        df_tfidf[target] = data[target]
        # df_tfidf['date'] = data['date']

        # 6. save tf-idf
        # pickle.dump(tfidf_transformer, open("tfidf.pkl","wb"))
        return df_tfidf

    # Data를 X, y로 나누어 return -> 추후 모델 학습 시 용이한 형태
    def data_transformation(self, data, target):
        # transform NA to 0
        data.fillna(0, inplace=True)
        
        # make date 0-1
        # data['date'] = data['date']/30000000
        
        # create a feature matrix
        X = data.drop(target, axis=1)

        # create a target vector
        y = data[target]
        
        # return the feature matrix and target vector
        return X, y

In [3]:
# # write a function to perform data exploration
# def perform_data_exploration(file_with_path):
    
#     # create an object of DataExploration class
#     data_exploration = DataExploration()

#     # load data HR_comma_sep.csv
#     data = data_exploration.load_data(file_with_path)

#     # Perform exploration
#     data_exploration.data_exploration(data)
    
#     # explore data
#     data_exploration.data_visualization(data)
    
#     return data

위에서 만든 클래스를 기반으로 데이터를 전처리하는 함수를 구현합니다.

In [4]:
# write a function to perform data preprocessing
def perform_data_preprocessing(data, target, train=False):
    # use DataPreprocessing class to perform data preprocessing
    # create an object of DataPreprocessing class
    data_preprocessing = DataPreprocessing()

    # data_preprocessing.get_writer_score(data, train)
    # new_data = data_preprocessing.merge_writer_score(data) # 분석 시 writer_score를 사용하는 경우
    new_data = data_preprocessing.title_tf_idf(data, target)

    # perform data_transformation
    #data_preprocessing.title_transformation(data)
    X, y = data_preprocessing.data_transformation(new_data, target)

    return  X, y

def data_splitting(X, y):
    # split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X,
                                                        y,
                                                        test_size=0.2,
                                                        random_state=1004)

    # return the training and testing sets
    return X_train, X_test, y_train, y_test

### __사용 예시__

In [5]:
df_train = pd.read_excel('../data/train_data.xlsx')
df_test = pd.read_excel('../data/news_20230308.xlsx')
df_test['tag'] = -1

FileNotFoundError: [Errno 2] No such file or directory: '../data/train_data.xlsx'

In [ ]:
df_train.head()

,docID,date,writerName,title,content,section,tag
0,1,20230214,머니S,"정부, AI반도체 석·박사 집중 육성… 대학당 '6년간 164억원' 지원",정부가 미국 AI 개발업체인 '오픈AI'(OpenAI)의 '챗GPT'(ChatGPT...,IT/과학,0
1,2,20230215,뉴시스,인사 청탁 대가 금품수수 의혹 전 소방청장 영장 기각,"기사내용 요약 법원 ""피의 사실 일부 다툼 여지, 불구속 상태 방어권 보장 필요"" ...",사회,0
2,3,20230214,아이뉴스24,튀르키예 강진에 우리나라 지하수가 출렁였다,튀르키예에서 발생한 강진에 우리나라의 지하수가 출렁였다는 관측 보고가 나왔다. 한국...,IT/과학,0
3,4,20230215,데일리안,"멸치쇼핑, 2023년 신입 및 경력 사원 대규모 공채 진행",[데일리안 = 박영민 기자] 오픈마켓 멸치쇼핑이 2023년 신입 및 경력 사원을 대...,생활/문화,0
4,5,20230111,뉴스1,"美국방부, 추모의 벽 전사자 명단 오류에 ""유감스러운 실수""","국방부 대변인 ""실수 바로잡기 위해 내무부와 협력""…'오류 발견' 가족 등에 연락 ...",정치외교,1


- 학습할 데이터와 테스트할 데이터가 한 파일 내에 있다면 train=True로 진행하면 됩니다.
- 학습 데이터와 테스트 데이터가 분리된 파일로 존재한다면 train=False로 진행하면 됩니다.

In [ ]:
## TRAIN & TEST SET

train = True  # train set으로만 결과 확인 시 True

df = pd.concat([df_train, df_test], ignore_index=True)
X, y = perform_data_preprocessing(data=df, target='tag', train=train)

if train:
    X, y = perform_data_preprocessing(data=df_train, target='tag', train=train)
    X_train, X_test, y_train, y_test = data_splitting(X, y)
else:
    df = pd.concat([df_train, df_test], ignore_index=True)
    X, y = perform_data_preprocessing(data=df, target='tag', train=train)   
    X_train = X[:len(df_train)]; X_test = X[len(df_train):]
    y_train = y[:len(df_train)]; y_test = y[len(df_train):]

## __Modeling__

In [ ]:
from sklearn.linear_model import LogisticRegression

# Train logistic regression model
classifier = LogisticRegression(random_state=42)
classifier.fit(X_train, y_train)

predicted = classifier.predict(X_test)
predicted_prob = classifier.predict_proba(X_test)

if train :
    result_nb = pd.DataFrame({'true_labels':y_test, 'predicted_labels':predicted})
    print('Logistic Regression')
    get_scores(y_test, predicted)

In [ ]:
# Multinomial Naive Bayes

from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB().fit(X_train, y_train)
#pickle.dump(clf, open("svm.pkl", "wb"))

#SAVE MODEL
#pickle.dump(clf, open("nb_model.pkl", "wb"))

predicted = clf.predict(X_test)
predicted_prob = clf.predict_proba(X_test)

if train:
    result_nb = pd.DataFrame({'true_labels':y_test, 'predicted_labels':predicted})

    print('Naive_Bayes')
    get_scores(y_test, predicted)

In [ ]:
from sklearn.neural_network import MLPClassifier

clf_neural = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(128,), max_iter=10000, random_state=1)
clf_neural.fit(X_train, y_train)
pickle.dump(clf_neural, open("softmax.pkl", "wb"))

predicted = clf_neural.predict(X_test)
predicted_prob = clf_neural.predict_proba(X_test)



if train:
    result_svm = pd.DataFrame({'true_labels':y_test, 'predicted_labels':predicted})

    print('Softmax')
    get_scores(y_test, predicted)
else:
    df_proba = pd.DataFrame(predicted_prob).rename(columns={0:'prob_0', 1:'prob_1'})
    df_test['tag'] = predicted
    df_test['prob_0'] = df_proba['prob_0']
    df_test['prob_1'] = df_proba['prob_1']
    
    # df_test.to_csv('../data/result_ksh_2.csv', index=False)

In [ ]:
from sklearn import svm

clf_svm = svm.LinearSVC()
clf_svm.fit(X_train, y_train)

#pickle.dump(clf_svm, open("svm.pkl", "wb"))

predicted = clf_svm.predict(X_test)

if train:
    result_svm = pd.DataFrame({'true_labels':y_test, 'predicted_labels':predicted})

    print('SVM')
    get_scores(y_test, predicted)

In [ ]:
from sklearn.calibration import CalibratedClassifierCV
from sklearn import svm

clf_svm = svm.LinearSVC()
clf = CalibratedClassifierCV(clf_svm, method='sigmoid') 
clf.fit(X_train, y_train)
y_proba = clf.predict_proba(X_test)

#pickle.dump(clf_svm, open("svm.pkl", "wb"))

predicted = clf.predict(X_test)

if train:
    result_svm = pd.DataFrame({'true_labels':y_test, 'predicted_labels':predicted})

    print('SVM')
    get_scores(y_test, predicted)
else:
    df_proba = pd.DataFrame(y_proba).rename(columns={0:'prob_0', 1:'prob_1'})
    df_test['tag'] = predicted
    df_test['prob_0'] = df_proba['prob_0']
    df_test['prob_1'] = df_proba['prob_1']
    
    df_test.to_csv('../data/result_tag.csv', index=False)

In [ ]:
df_test['tag'].value_counts()

0    5945
1    1426
Name: tag, dtype: int64

In [ ]:
df_test['section'].value_counts()

사회       3509
경제       1774
정치       1129
세계        357
생활/문화     328
IT/과학     274
Name: section, dtype: int64

In [ ]:
df_test[df_test['section']=='정치']['tag'].value_counts()

0    579
1    550
Name: tag, dtype: int64

In [ ]:
df_test[df_test['tag']==1]['section'].value_counts()

정치       550
사회       486
경제       195
세계       157
IT/과학     20
생활/문화     18
Name: section, dtype: int64

In [ ]:
df_test[df_test['section']=='세계']

,docID,date,writerName,title,content,section,tag,prob_0,prob_1
10,202303080030011730048,20230308,뉴시스,"WBC 대한민국 vs 호주, 기자회견하는 양팀 감독들",[도쿄=뉴시스] 김선웅 기자 = WBC 대한민국 야구대표팀 이강철 감독과 호주대표팀...,세계,0,0.577398,0.422602
12,202303080030011729801,20230308,뉴시스,"이강철 야구대표팀 감독, 기자회견",[도쿄=뉴시스] 김선웅 기자 = WBC 대한민국 야구대표팀 이강철 감독이 8일 일본...,세계,0,0.936086,0.063914
13,202303080030011729793,20230308,뉴시스,밝은 표정으로 기자회견하는 이강철 감독,[도쿄=뉴시스] 김선웅 기자 = WBC 대한민국 야구대표팀 이강철 감독이 8일 일본...,세계,0,0.864316,0.135684
14,202303080030011729791,20230308,뉴시스,기자회견하는 김현수,[도쿄=뉴시스] 김선웅 기자 = WBC 대한민국 야구대표팀 주장 김현수가 8일 일본...,세계,0,0.720959,0.279041
22,202303084210006671416,20230308,뉴스1,기자회견 참석한 中 외교부장,(베이징 AFP=뉴스1) 김성식 기자 = 친강 중국 외교부장이 7일 중국 베이징 전...,세계,1,0.024120,0.975880
...,...,...,...,...,...,...,...,...,...
7347,202303084210006670227,20230308,뉴스1,"파월 ""최종 금리 수준, 이전 예상보다 더 높을 가능성 커""(1보)",(서울=뉴스1) 김민수 기자 = 제롬 파월 연방준비제도(연준) 의장은 7일(현지시간...,세계,1,0.270165,0.729835
7357,202303080250003264218,20230308,중앙일보,[사진] “대만은 중국 영토” 헌법 읽는 친강 외교부장,친강 중국 외교부장이 7일 취임 후 첫 기자회견에서 대만 관련 질문이 나오자 붉은 ...,세계,1,0.009092,0.990908
7363,202303080150004818282,20230308,한국경제,"백악관 ""윤 대통령, 4월26일 국빈 방문…동맹 70년 기념"" [종합]",7일(현지시간) 미국 백악관은 윤석열 대통령이 다음 달 26일 미국을 국빈 방문한다...,세계,1,0.000988,0.999012
7366,202303080030011728700,20230308,뉴시스,방글라데시 다카서 상가 건물 폭발…최소 14명 사망,기사내용 요약 50명 이상 부상…폭발 충격에 건물 1~2층 크게 파손 [다카(방글라...,세계,0,0.892633,0.107367


In [ ]:
df_tag_1 = df_test[df_test['tag']==1]
df_tag_1.to_csv('../data/result_tag1.csv', index=False)